## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

1)

In [ ]:
try:
    data = pd.read_csv("data_000637.txt")
except:
    !wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
    data = pd.read_csv("data_000637.txt")    
    
data.info()
data.head()

2)

In [ ]:
print("The number of BX in an orbit is:", data.groupby("ORBIT_CNT").max()["BX_COUNTER"].max()+1) # to account for zero


4-3)

In [ ]:
data["abs_time (ns)"] = data['TDC_MEAS']*25/30+data['BX_COUNTER']*25+data["ORBIT_CNT"]*3564*25
print('time taken for data acquisition: (ns)', data["abs_time (ns)"].iat[-1]-data['abs_time (ns)'].iat[0])


5)

In [ ]:
data["HEAD"] = np.random.choice([0,1],size=(len(data["HEAD"]),))
data.head()

6)

In [ ]:
data_new = data[data["HEAD"]==1]
data_new.info()

7)

In [ ]:
axes = data_new.groupby(["FPGA", "TDC_CHANNEL"])["HEAD"].count().unstack('FPGA').plot(subplots=True,kind='bar',title=["FPGA 0", "FPGA 1"],legend=False,sharex=False,figsize=(10,15));

for ax in axes:
    for i,t in enumerate(ax.get_xticklabels()):
        if not (i % 5) == 0:
            t.set_visible(False)




8)

In [ ]:
print("TDC Channels with more counts (top 3):", data_new.groupby("TDC_CHANNEL").count().sort_values(by="HEAD",ascending=False).index[:3].values)

9)

In [ ]:
print("Number of unique orbits", data_new["ORBIT_CNT"].unique().shape[0])
print("number of unique orbits with at least one measurement from TDC_CHANNEL=139",data_new[data_new["TDC_CHANNEL"] == 139]["ORBIT_CNT"].unique().shape[0])
